https://github.com/rssalessio/PytorchRBFLayer

# 3-class classification example

In this example we try to learn a model that is able to classify 3 classes drawn from 3 different normal distributions

In [3]:
import os
import sys
sys.path.append("..")
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D  
from rbf_layer.rbf_layer import RBFLayer
from rbf_layer.rbf_utils import l_norm, rbf_gaussian

# Dataset Generation

In [4]:
# # Generate dataset
# N_training = 100
# N_validation = 50
# N = N_training + N_validation
#
#
# X0 = np.array([-1, 0]).T + 0.5 * np.random.normal(size=(N, 2))
# X1 = np.array([1, 0]).T +  0.5 * np.random.normal(size=(N, 2))
# X2 = np.array([0, 1]).T + 0.3 * np.random.normal(size=(N, 2))
# X = np.concatenate([X0, X1, X2])
#
# labels = np.zeros((3*N, 1))
# labels[N:2*N] = 1
# labels[2*N:] = 2
#
# # Training dataset
# indices = np.random.permutation(3 * N)
# trn_indices, val_indices = indices[:N_training], indices[N_training:]
# Tdataset = ([torch.tensor(X[trn_indices], dtype=torch.float32), torch.tensor(labels[trn_indices], dtype=torch.float32)])
# Vdataset = ([torch.tensor(X[val_indices], dtype=torch.float32), torch.tensor(labels[val_indices], dtype=torch.float32)])
#
# # Plot dataset
# plt.scatter(X0[:, 0], X0[:, 1])
# plt.scatter(X1[:, 0], X1[:, 1])
# plt.scatter(X2[:, 0], X2[:, 1])
# plt.grid()
# plt.title('Feature space')
# plt.xlabel('Feature 1')
# plt.ylabel('Feature 2')
# plt.plot()

In [5]:
import pathlib
from subgraph_matching_via_nn.data.data_loaders import load_graph
import os

goal_subcircuit_name = 'alu'
DATA_PATH = 'C:/Users/kogan/OneDrive/Desktop/Research/AMIT/GraphMatching/subgraph_matching_via_nn/data/subcircuits/'
desktop = pathlib.Path(DATA_PATH)
subgraphs = []
labels = []
for circuit_dir in desktop.iterdir():
    if circuit_dir.is_dir():
        for subcircuit_file in circuit_dir.iterdir():
            if subcircuit_file.is_file():
                file_name = subcircuit_file.name
                if file_name == 'full_graph.p':
                    file_rel_path = f"{os.sep}{file_name}"
                    loader_params = {
                     'data_path' : str(circuit_dir),
                     'g_full_path': file_rel_path,
                     'g_sub_path': file_rel_path}

                    sub_graph = \
                        load_graph(type='subcircuit',
                                   loader_params=loader_params)

                    if goal_subcircuit_name in circuit_dir.name:
                        labels.append(1)
                    else:
                        # continue #TODO?
                        labels.append(0)
                    subgraphs.append(sub_graph)

labels = np.array(labels, dtype='float32')
N = len(subgraphs)
N_training = int(2 / 3 * N)

In [6]:
subgraphs = []
labels = []

circuit_base_dir = 'C:\\Users\\kogan\\OneDrive\\Desktop\\Research\\AMIT\\GraphMatching\\subgraph_matching_via_nn\\data\\subcircuits\\'
for circuit_file_name in ['adder_4', 'alu_4', 'alu_8', 'alu_16', 'alu_32', 'mul_4_4', 'mul_4_8', 'mul_8_8', 'mul_16_32']:
    file_rel_path = 'full_graph.p'
    circuit_dir = f"{circuit_base_dir}{circuit_file_name}{os.sep}"
    loader_params = {
     'data_path' : str(circuit_dir),
     'g_full_path': file_rel_path,
     'g_sub_path': file_rel_path}

    sub_graph = \
        load_graph(type='subcircuit',
                   loader_params=loader_params)
    subgraphs.append(sub_graph)
    labels.append(0)
labels[0] = 1

In [7]:
N = len(subgraphs)
N_training = int(2 / 3 * N)

In [8]:
len(subgraphs)

9

In [9]:
len(labels)

9

In [10]:
def prepare_x_and_labels(subgraphs, labels, indices, N_training):
    trn_indices, val_indices = indices[:N_training], indices[N_training:]

    trn_x = [subgraphs[ind] for ind in trn_indices]
    trn_label = [labels[ind] for ind in trn_indices]
    validation_x = [subgraphs[ind] for ind in val_indices]
    validation_label = [labels[ind] for ind in val_indices]

    for ind in indices:
        if labels[ind] == 1:
            break
    print(ind)
    compared_subgraph = subgraphs[ind]

    return trn_x, trn_label, validation_x, validation_label, compared_subgraph

# Training dataset
indices = np.random.permutation(N)
trn_x, trn_label, validation_x, validation_label, compared_subgraph = prepare_x_and_labels(subgraphs, labels, indices, N_training)

0


In [11]:
N_training = N
trn_x = validation_x = subgraphs
trn_label = validation_label = labels

In [12]:
labels

[1, 0, 0, 0, 0, 0, 0, 0, 0]

In [13]:
validation_label

[1, 0, 0, 0, 0, 0, 0, 0, 0]

In [14]:
N_training

9

In [15]:
# for subgraph in subgraphs:
#     print(rbf_graph_model(subgraph))

# for id, label in enumerate(labels):
#     if label == 1:
#         print(id)

# Define full model

In [17]:
from notebooks.rbf_metric import GCNEmbeddingNetwork, RBFGraphModel

embedding_features_size = 3
embedding_model = GCNEmbeddingNetwork(1, embedding_features_size)

rbf_graph_model = RBFGraphModel(embedding_model, embedding_features_size)

# embedding_model = GCNEmbeddingNetwork(1, 3)
# some_A = subgraphs[0].A_full
# res = embedding_model(some_A)

# Training code

In [18]:
from torch.cuda.amp import custom_bwd, custom_fwd

class DifferentiableClamp(torch.autograd.Function):
    """
    In the forward pass this operation behaves like torch.clamp.
    But in the backward pass its gradient is 1 everywhere, as if instead of clamp one had used the identity function.
    """

    @staticmethod
    @custom_fwd
    def forward(ctx, input, min, max):
        return input.clamp(min=min, max=max)

    @staticmethod
    @custom_bwd
    def backward(ctx, grad_output):
        return grad_output.clone(), None, None


def dclamp(input, min, max):
    """
    Like torch.clamp, but with a constant 1-gradient.
    :param input: The input that is to be clamped.
    :param min: The minimum value of the output.
    :param max: The maximum value of the output.
    """
    return DifferentiableClamp.apply(input, min, max)

# rbf_graph_model.train(mode=True)

def calc_loss(x, labels):
    #TODO: calc distance vs trn_x[0]
    compared_embedding = rbf_graph_model(compared_subgraph).to(dtype=torch.float64)
    y = [rbf_graph_model(x_).to(dtype=torch.float64) for x_ in x]
    y = [torch.nn.MSELoss()(y_, compared_embedding) for y_ in y]

    predictions = torch.stack(y)
    # predictions = torch.cat(y, dim=0).squeeze(1)
    # predictions = dclamp(predictions, 0, 1)
    predictions = torch.sigmoid(predictions)#.reshape(-1)

    targets = torch.tensor(labels, dtype=torch.float64)

    try:
        loss = nn.BCELoss()(predictions, targets)
    except RuntimeError as e:
        print(e)
        print(predictions)
        print("BUG")
    return loss

optimiser = torch.optim.Adam(rbf_graph_model.parameters(), lr=5e-6)
# optimiser = torch.optim.SGD(rbf_graph_model.parameters(), lr=1e-6, momentum=0.9, weight_decay=5e-1)
epochs_number = 2000
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimiser, T_max=epochs_number)
epoch = 0
batch_size = N_training
trn_losses = []
val_losses = []

for epoch in range(2000):
    indices = np.random.permutation(N_training)
    batch_idx = 0
    epoch_trn_losses = []
    epoch_val_losses = []

    # Epoch training
    while batch_idx < N_training:
        idxs = indices[batch_idx:batch_idx + batch_size]
        x = [trn_x[idx] for idx in idxs]
        labels = [trn_label[idx] for idx in idxs]
        
        # Compute loss
        optimiser.zero_grad()
        loss = calc_loss(x, labels)
        epoch_trn_losses.append(loss.item())
        loss.backward()

        # for name, param in rbf_graph_model.named_parameters():
        #     print(name, param.grad)
        optimiser.step()
        batch_idx += batch_size
        
        with torch.no_grad():
            # Compute validation
            loss = calc_loss(validation_x, validation_label)

            epoch_val_losses.append(loss.item())
    print(f"\nepoch {epoch} loss={loss.item()}")

    trn_losses.append(np.mean(epoch_trn_losses))
    val_losses.append(np.mean(epoch_val_losses))

    if (epoch + 1) % 100 == 0:
        if scheduler is not None:
            scheduler.step()

plt.plot(trn_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend()
plt.grid()
plt.title('Training plot')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()


epoch 0 loss=48.59091430252944

epoch 1 loss=48.503508748313294

epoch 2 loss=42.26427682369152

epoch 3 loss=47.56667860723412

epoch 4 loss=47.546267710323285

epoch 5 loss=58.50940748710554

epoch 6 loss=39.994474375029526

epoch 7 loss=41.78227340425431

epoch 8 loss=59.93453624303996

epoch 9 loss=47.95115553410864

epoch 10 loss=51.68790844652606

epoch 11 loss=60.08945792782217

epoch 12 loss=48.35503865349827

epoch 13 loss=46.759260303813406

epoch 14 loss=50.54887908000389

epoch 15 loss=41.19282300137268

epoch 16 loss=48.747275470465404

epoch 17 loss=59.22604486916688

epoch 18 loss=49.98732204259607

epoch 19 loss=60.83136632932004

epoch 20 loss=38.94177680648993

epoch 21 loss=49.626424867658606

epoch 22 loss=61.54340633169467

epoch 23 loss=52.049920349306916

epoch 24 loss=60.02195679685182

epoch 25 loss=48.30190497046791

epoch 26 loss=57.464835375189296

epoch 27 loss=41.621472274940544

epoch 28 loss=50.7130088408924

epoch 29 loss=49.471212358641914

epoch 30 l

KeyboardInterrupt: 

In [19]:
validation_label

[1, 0, 0, 0, 0, 0, 0, 0, 0]

In [20]:
rbf_graph_model.eval()
compared_embedding = rbf_graph_model(compared_subgraph).to(dtype=torch.float64)
y = [rbf_graph_model(x_).to(dtype=torch.float64) for x_ in subgraphs]
y = [torch.nn.MSELoss()(y_, compared_embedding) for y_ in y]

In [21]:
labels

[0, 1, 0, 0, 0, 0, 0, 0, 0]

In [22]:
for ind, label in enumerate(labels):
    if label == 1:
        print(ind)

1


In [23]:
y

[tensor(0., dtype=torch.float64, grad_fn=<MseLossBackward>),
 tensor(0.6836, dtype=torch.float64, grad_fn=<MseLossBackward>),
 tensor(28.6607, dtype=torch.float64, grad_fn=<MseLossBackward>),
 tensor(207.9005, dtype=torch.float64, grad_fn=<MseLossBackward>),
 tensor(1060.0068, dtype=torch.float64, grad_fn=<MseLossBackward>),
 tensor(0.6568, dtype=torch.float64, grad_fn=<MseLossBackward>),
 tensor(13.8747, dtype=torch.float64, grad_fn=<MseLossBackward>),
 tensor(44.5584, dtype=torch.float64, grad_fn=<MseLossBackward>),
 tensor(900.3855, dtype=torch.float64, grad_fn=<MseLossBackward>)]

## Plot kernels

In [ ]:
# kernels = rbf.get_kernels_centers.numpy()
# shapes = rbf.get_shapes.numpy()
#
# def fun(x, y, center, shape):
#     diff = center.T - [x, y]
#     r = np.linalg.norm(diff, axis=0)
#     return np.exp(-(shape * r) ** 2)
#
# x = y = np.arange(-2.5, 2.5, 0.05)
# X, Y = np.meshgrid(x, y)
# fig, ax = plt.subplots()
#
# for i in range(3):
#     center = kernels[i][:, None].repeat(10000, axis=1).T
#     zs = np.array(fun(X.ravel(), Y.ravel(), center, shapes[i].repeat(10000)))
#     Z = zs.reshape(X.shape)
#
#     CS = ax.contour(X, Y, Z)
#     ax.clabel(CS, inline=True, fontsize=10)
# ax.set_title('Learnt kernels')
# ax.set_xlabel('Feature 1')
# ax.set_ylabel('Feature 2')
# ax.grid()
# plt.show()

# Plot decision boundary

In [ ]:
# x = y = np.arange(-2.5, 2.5, 0.05)
# X, Y = np.meshgrid(x, y)
#
# # Compute model prediction
# inp = torch.tensor([X.ravel(), Y.ravel()]).T
# Z = rbf(inp).argmax(dim=1).detach().numpy()
#
# # Put the result into a color plot
# Z = Z.reshape(X.shape)
# plt.contourf(X, Y, Z, cmap=cm.Pastel1)
#
# # Plot dataset
# plt.scatter(X0[:, 0], X0[:, 1], c='r')
# plt.scatter(X1[:, 0], X1[:, 1], c='m')
# plt.scatter(X2[:, 0], X2[:, 1], c='gray')
#
# plt.title('Decision boundaries')
# plt.xlabel('Feature 1')
# plt.ylabel('Feature 2')
# plt.plot()
# plt.savefig('example_classification_img.png', bbox_inches='tight')

apply KDE

In [ ]:
from sklearn.naive_bayes import GaussianNB

#TODO
# https://github.com/sampath9dasari/NaiveBayesClassifier-KDE/blob/master/NaiveBayes_with_KDE.ipynb

x_train = [embedding_model(sample.A_full).detach().numpy() for sample in trn_x]
y_train = trn_label
x_test = [embedding_model(sample.A_full).detach().numpy() for sample in validation_x]
y_test = validation_label

gaussian_model = GaussianNB()
gaussian_model.fit(x_train,y_train)

In [ ]:
gaussian_model.score(x_test, y_test)

In [ ]:
gaussian_model.predict(x_test)

In [ ]:
# enhance dataset: can append distances to the embedding (the distance from the graph) for graphs which we know should not be similar
#TODO
